In [1]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import seaborn
import joblib
from utils import load_data

In [2]:
# Avoid tensorflow warnings and info messages about my poor bad CPU
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

**Deep learning part**

Note: See [here](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) for dataset directory structure.

In [3]:
#Path and names
X_training_set_path = "training_set/X_training.npy"
y_training_set_path = "training_set/y_training.npy"
X_validation_set_path = "validation_set/X_validation.npy"
y_validation_set_path = "validation_set/y_validation.npy"

In [4]:
# Model & data parameters
model_name = 'retrain_mobilenet_v4'
retrain_convolution = False
num_classes = 50

image_size = 192 #in pixels
num_classes = 50
validation_size = 0.2
input_shape = (image_size, image_size, 3)

In [5]:
# Training parameters
optimizer = keras.optimizers.Adam(1e-4) #learning_rate=0.001 (default value)
epochs = 50
batch_size = 16

**Load data from dataset images**

In [ ]:
#Load dataset and normalize data to the range [-1, 1]
X, y = load_data((image_size, image_size))
X /= 127.5
X -= 1
# Split Training/Testing and validation test
X, X_validation, y, y_validation = train_test_split(X, y, test_size=validation_size)

**Save dataset for next time**

In [ ]:
if not os.path.exists(training_set_path):
    os.makedirs(training_set_path)
    
if not os.path.exists(validation_set_path):
    os.makedirs(validation_set_path)
    
np.save(X_training_set_path, X)
np.save(y_training_set_path, y)
np.save(X_validation_set_path, X_validation)
np.save(y_validation_set_path, y_validation)

**Load data from already saved numpy array**

In [ ]:
X = np.load(X_training_set_path)
y = np.load(y_training_set_path)
X_validation = np.load(X_validation_set_path)
y_validation = np.load(y_validation_set_path)

In [6]:
# Call back
#early_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [7]:
# Tensorflow data augmentation
datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=20,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

datagen.fit(X)

In [ ]:
# MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

pretrain = MobileNetV2(weights="imagenet",  alpha=0.5, input_shape = input_shape, include_top = False)

In [9]:
# ResNet
from tensorflow.keras.applications.resnet import ResNet101

pretrain = ResNet101(weights="imagenet", input_shape = input_shape, include_top = False)

In [11]:
# DenseNet
from tensorflow.keras.applications.densenet import DenseNet201

pretrain = DenseNet201(weights="imagenet", input_shape = input_shape, include_top = False)

In [ ]:
# Custom Network

M = keras.Sequential()

M.add(layers.Conv2D(32, (3, 3), input_shape = input_shape, activation = 'relu'))
M.add(layers.MaxPooling2D(pool_size = (2, 2)))
M.add(layers.Conv2D(32, (3, 3), activation = 'relu'))
M.add(layers.MaxPooling2D(pool_size = (2, 2)))
M.add(layers.Flatten())

M.add(layers.Dense(64, activation = 'relu'))
M.add(layers.Dense(num_classes, activation = 'softmax'))
model = M
model.summary()

In [ ]:
# Init the selected model (if not a custom one, else do not run this part)
if not retrain_convolution:
    for layer in pretrain.layers:
        layer.trainable = False
    pretrain.layers[0].trainable = False
    
pretrain_out = pretrain.output

M = layers.MaxPooling2D()(pretrain_out)
M = layers.Flatten()(M)
M = layers.Dropout(0.5)(M)
M = layers.Dense(num_classes, activation="softmax")(M)

#Compile the model
model = keras.Model(inputs=pretrain.input, outputs=M)
model.summary()

In [ ]:
# Training model without tensorflow data augmentation
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", "top_k_categorical_accuracy"])
history = model.fit(X, y, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)

In [ ]:
# Training model with tensorflow data augmentation
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", "top_k_categorical_accuracy"])
history = model.fit(
         datagen.flow(X, y, batch_size=batch_size, subset='training'),
         validation_data=datagen.flow(X, y,batch_size=16, subset='validation'),
         epochs=epochs, 
         verbose=1)

In [ ]:
# Validation of the model
scores = model.evaluate(X_validation, y_validation, verbose=1)

In [ ]:
# Predict validation set
prediction = np.argmax(model.predict(X_validation), axis=-1)
y_prediction = np.argmax(y_validation, axis=-1)

# Confusion matrix of this validation
cm = metrics.confusion_matrix(y_prediction, prediction)
plt.figure(figsize = (10,7))
seaborn.heatmap(cm, annot=True, linewidths=1)
plt.savefig("dl_confusion_matrix")

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Retrain " + model_name + " (alpha= 0.5) top 1 accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper left")
plt.show()

In [ ]:
# Plot training & validation top 5 values
plt.plot(history.history['top_k_categorical_accuracy'])
plt.plot(history.history['val_top_k_categorical_accuracy'])
plt.title("Retrain " + model_name + " (alpha= 0.5) top 5 accuracy")
plt.ylabel("Top 5 categorical accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper left")
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Retrain " + model_name + " (alpha= 0.5) loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper left")
plt.show()

In [17]:
# Save the model
model_path = './models/' + model_name + '.keras'
model.save(model_path)

**Set features extractor for Extremely Randomized Trees (ET) and Support Vector Machine (SVM)**

In [3]:
# Path and names
dl_model_name = "retrain_mobilenet_v4"

# Layers to remove from our deep learning model to only get convolutif part
layers_to_remove = 3

In [4]:
# Load the deep learning model
model = keras.models.load_model('./models/' + dl_model_name + '.keras')

In [5]:
# Output of the CNN (without the dense part)
feature_extractor = keras.Model(model.input, model.layers[-layers_to_remove].output)

**Extremely Randomized Trees & Support Vector Machine model** <br>
This part is for add a model after our convolution

- 1. We train our model on the same training set
- 2. Test accuracy on the same validation set

In [6]:
# Parameters
analyzer_path = "./analyzer_model.joblib"
n_estimators = 225

def transform_one_hot_labels(one_hot_labels):
    y = []
    for o in one_hot_labels:
        y.append(np.argmax(o))
    return y

**1. Training part**

In [7]:
# Load the training set
X = np.load(X_training_set_path)
y = np.load(y_training_set_path)

# Labels need to be indice of the classes as target value, not one hot encoded
y = transform_one_hot_labels(y)

**Extremely Randomized Trees**

In [8]:
# Init Extra trees mdoel
model = ExtraTreesClassifier(n_estimators = n_estimators)

**Support Vector Machines**

In [ ]:
# Init the SVM model
model = SVC()

**Extract features and train**

In [ ]:
# Extract features
X = feature_extractor.predict(X)

In [9]:
# Train the model
model.fit(X, y)

ExtraTreesClassifier(n_estimators=225)

In [10]:
# Save the random forest model
joblib.dump(model, analyzer_path)

['./random_forest.joblib']

**2. Validation part**

In [7]:
# Load the validation set
X_validation = np.load(X_validation_set_path)
y_validation = np.load(y_validation_set_path)

# Labels need to be indice of the classes as target value, not one hot encoded
y_validation = transform_one_hot_labels(y)

In [10]:
# Load the analyzer model (if needed)
model = joblib.load(analyzer_path)

In [ ]:
# Test on validation test
X_validation_features = feature_extractor(X_validation)
predicted_values = model.predict(X_validation_features)

In [ ]:
# Accuracy
print("Top 1 accuracy: ", metrics.accuracy_score(y_validation, predicted_values))

In [ ]:
# Confusion matrix
cm = metrics.confusion_matrix(y_validation_encoded, predicted_values)
seaborn.heatmap(cm, annot=True, linewidths=1)
plt.savefig("DL&AnalyzerModel_confusion_matrix")